In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [2]:
# Пример текстов и меток
texts = ["I love this movie", "This movie is bad", "I really enjoyed this film", "Worst movie ever", "Great acting", "Terrible plot"]
labels = [1, 0, 1, 0, 1, 0]  # 1 - позитивный отзыв, 0 - негативный

In [3]:
# Векторизация текста
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(texts).toarray()

# Разделим на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [4]:
# Преобразуем данные в тензоры
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [5]:
# Создаем датасеты и загрузчики данных
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [ ]:
# Создадим простую нейронную сеть с одним скрытым слоем для классификации текста.

In [6]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Скрытый слой
        self.fc2 = nn.Linear(hidden_size, output_size)  # Выходной слой
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Применяем ReLU к первому слою
        x = self.fc2(x)  # Пропускаем через второй слой
        return self.softmax(x)  # Softmax для многоклассовой классификации (или 2 класса)


In [7]:
# Параметры модели
input_size = X_train.shape[1]  # Размерность входных данных
hidden_size = 10  # Количество нейронов в скрытом слое
output_size = 2  # Два класса (позитивный и негативный)

# Инициализация модели
model = SimpleNN(input_size, hidden_size, output_size)

# Определим функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()  # Функция потерь для классификации
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Обнуляем градиенты
        outputs = model(inputs)  # Получаем прогнозы
        loss = criterion(outputs, labels)  # Вычисляем потерю
        loss.backward()  # Обратный проход
        optimizer.step()  # Шаг оптимизации

        running_loss += loss.item()

        # Преобразуем выходные данные в классы
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {correct/total:.4f}")


Epoch [1/20], Loss: 0.6959, Accuracy: 0.5000
Epoch [2/20], Loss: 0.6945, Accuracy: 0.5000
Epoch [3/20], Loss: 0.6934, Accuracy: 0.5000
Epoch [4/20], Loss: 0.6918, Accuracy: 0.5000
Epoch [5/20], Loss: 0.6905, Accuracy: 0.5000
Epoch [6/20], Loss: 0.6895, Accuracy: 0.5000
Epoch [7/20], Loss: 0.6880, Accuracy: 0.5000
Epoch [8/20], Loss: 0.6870, Accuracy: 0.5000
Epoch [9/20], Loss: 0.6855, Accuracy: 0.5000
Epoch [10/20], Loss: 0.6842, Accuracy: 0.5000
Epoch [11/20], Loss: 0.6830, Accuracy: 0.5000
Epoch [12/20], Loss: 0.6816, Accuracy: 0.5000
Epoch [13/20], Loss: 0.6804, Accuracy: 0.5000
Epoch [14/20], Loss: 0.6791, Accuracy: 0.5000
Epoch [15/20], Loss: 0.6778, Accuracy: 0.5000
Epoch [16/20], Loss: 0.6767, Accuracy: 0.5000
Epoch [17/20], Loss: 0.6751, Accuracy: 0.5000
Epoch [18/20], Loss: 0.6738, Accuracy: 0.5000
Epoch [19/20], Loss: 0.6725, Accuracy: 0.7500
Epoch [20/20], Loss: 0.6713, Accuracy: 1.0000


In [8]:
# Оценка модели

In [9]:
# Оценка модели на тестовой выборке
model.eval()  # Переводим модель в режим оценки
correct = 0
total = 0
with torch.no_grad():  # Не нужно вычислять градиенты
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {correct/total:.4f}")


Test Accuracy: 0.5000
